In [1]:
from lxml import etree
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import tushare as ts
import tqdm
pro = ts.pro_api('5e2da4e6322b3b893f1c10e43d001749300351741d736d247251d837')

In [6]:
#批量更新 由于csv追加写入，不能乱运行。
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'}

with open('龙虎榜营业部.csv','a',encoding='gbk') as f:
    f.write('date,stock,营业部,买入(万),卖出(万),净买(万)\n')

df2 = pro.trade_cal(exchange='', start_date='20180101', end_date='20211202')
cal = df2[df2['is_open']==1]['cal_date'].tolist()
cal_format = list(map(lambda x:'{}-{}-{}'.format(x[:4],x[4:6],x[-2:]),cal))

for date in tqdm.tqdm_notebook(cal_format):
    with open('龙虎榜营业部.csv','a',encoding='gbk') as f:
        url = 'http://data.10jqka.com.cn/ifmarket/lhbggxq/report/%s/'%date
        html = requests.get(url=url,headers=headers)
        if html.status_code==200:
            soup = BeautifulSoup(html.text, 'lxml')
            elements = soup.find_all(attrs={'class':'stockcont'})
            for element in elements:
                tmp_text = element.find('p').text
                name,tmp_text2 = tmp_text.split(')')
                a = list(map(lambda x:x.find('a').get('title') if x.find('a') else x.text.replace('\n',''),element.find_all('td')))
                b = np.array(a).reshape(-1,4)
                length = b.shape[0]
                df = pd.DataFrame(b,index=[[date]*length,[name]*length],columns=['营业部','买入','卖出','净买'])
                df.index.names = ['date','stock']
                string = '\r\n'.join(df.to_csv().split('\r\n')[1:]).replace('\r','').replace('一线游资','')
                f.write(string)
        else:
            print('%s凉了'%date)

<ipython-input-6-2251dcbc650b>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for date in tqdm.tqdm_notebook(cal_format):


  0%|          | 0/952 [00:00<?, ?it/s]

2020-07-24凉了
2020-12-25凉了
2020-12-28凉了


In [11]:
#daily update 由于csv追加写入，不能乱运行。
date = input('yyyy-mm-dd:')
with open('龙虎榜营业部.csv','a',encoding='gbk') as f:
    url = 'http://data.10jqka.com.cn/ifmarket/lhbggxq/report/%s/'%date
    html = requests.get(url=url,headers=headers)
    print(html.status_code)
    if html.status_code==200:
        soup = BeautifulSoup(html.text, 'lxml')
        elements = soup.find_all(attrs={'class':'stockcont'})
        for element in elements:
            tmp_text = element.find('p').text
            name,tmp_text2 = tmp_text.split(')')
            a = list(map(lambda x:x.find('a').get('title') if x.find('a') else x.text.replace('\n',''),element.find_all('td')))
            b = np.array(a).reshape(-1,4)
            length = b.shape[0]
            df = pd.DataFrame(b,index=[[date]*length,[name]*length],columns=['营业部','买入','卖出','净买'])
            df.index.names = ['date','stock']
            string = '\r\n'.join(df.to_csv().split('\r\n')[1:]).replace('\r','').replace('一线游资','').replace('知名游资','')
            f.write(string)
    else:
        print('%s凉了'%date)

200


In [23]:
df = pd.read_csv('龙虎榜营业部.csv',index_col=[0,1],encoding='gbk')
no_yyb_list = ['深股通专用 ','机构专用','沪股通专用 ']
df.head()

营业部   买入(万)   卖出(万)   净买(万)
date       stock                                                       
2018-01-02 金岭矿业(000655  国泰君安证券股份有限公司成都北一环路证券...  612.19    0.00  612.19
           金岭矿业(000655   国泰君安证券股份有限公司北京方庄路证券营业部  328.52    0.00  328.52
           金岭矿业(000655    国信证券股份有限公司深圳泰然九路证券营业部  305.82  162.13  143.69
           金岭矿业(000655  国金证券股份有限公司上海奉贤区金碧路证券...  197.17   14.60  182.56
           金岭矿业(000655   国泰君安证券股份有限公司泸州星光路证券营业部  160.67    0.00  160.67

In [26]:
df_2020 = df.iloc[list(map(lambda x:x[0][:4]=='2020', df.index)),:]
df_2020_top = df_2020.groupby('营业部').sum().sort_values('买入(万)',ascending=False ).head(53)
df_2020_top_youzi = df_2020_top.iloc[~df_2020_top.index.isin(no_yyb_list)]
df_2020_top_youzi.sum()

买入(万)    55148070.16
卖出(万)    39927172.64
净买(万)    15220897.08
dtype: float64

In [27]:
df_2020 = df.iloc[list(map(lambda x:x[0][:4]=='2020', df.index)),:]
df_2020.groupby('营业部').sum().sort_values('买入(万)',ascending=False ).head(50).sum()

买入(万)    1.148171e+08
卖出(万)    9.053341e+07
净买(万)    2.428372e+07
dtype: float64

In [28]:
df_2021 = df.iloc[list(map(lambda x:x[0][:4]=='2021', df.index)),:]
df_2021.groupby('营业部').sum().sort_values('买入(万)',ascending=False ).head(50).sum()


买入(万)    1.713472e+08
卖出(万)    1.403663e+08
净买(万)    3.098097e+07
dtype: float64

In [29]:
df_2021 = df.iloc[list(map(lambda x:x[0][:4]=='2021', df.index)),:]
df_2021_top = df_2021.groupby('营业部').sum().sort_values('买入(万)',ascending=False ).head(53)
df_2021_top_youzi = df_2021_top.iloc[~df_2021_top.index.isin(no_yyb_list)]
df_2021_top_youzi.sum()

买入(万)    66502513.40
卖出(万)    49906592.32
净买(万)    16595920.03
dtype: float64